In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
file_path = "XAUUSD_H1_202201022300_202410252300.csv"
df = pd.read_csv(file_path, delimiter="\t")
df.columns = [col.strip('<>') for col in df.columns]
columns_to_use = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [4]:
scaler = MinMaxScaler()
df_filtered = df[columns_to_use]
df_scaled = scaler.fit_transform(df_filtered)

In [5]:
lookback_window = 50 
X, y = [], []
for i in range(len(df_scaled) - lookback_window - 10): 
    X.append(df_scaled[i:i+lookback_window])  
    y.append(df_scaled[i+lookback_window+10, columns_to_use.index('CLOSE')]) 

X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\Albaloo Rayaneh\AppData\Local\Temp\ipykernel_14612\646333479.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X = torch.tensor(X, dtype=torch.float32).to(device)


In [6]:
class LSTM_BiGRU_Model(nn.Module):
    def __init__(self):
        super(LSTM_BiGRU_Model, self).__init__()
        self.lstm = nn.LSTM(input_size=len(columns_to_use), hidden_size=64, num_layers=1, batch_first=True)
        self.bigru = nn.GRU(input_size=64, hidden_size=64, num_layers=1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(64 * 2, 1)  

    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.bigru(x)
        x = x[:, -1, :] 
        x = self.fc(x)
        return x

In [7]:
model = LSTM_BiGRU_Model().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) 

In [8]:
num_epochs = 50  
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_train_pred = model(X_batch)
        train_loss = criterion(y_train_pred.squeeze(), y_batch)
        train_loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_test_preds, y_tests = [], []
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_preds.append(y_test_pred.squeeze())
            y_tests.append(y_batch)
        
        y_test_pred = torch.cat(y_test_preds)
        y_test = torch.cat(y_tests)
        test_loss = criterion(y_test_pred, y_test)
    
    if (epoch + 1) % 10 == 0:
        r2_train = r2_score(y_train.cpu().detach().numpy(), model(X_train).cpu().detach().squeeze().numpy())
        r2_test = r2_score(y_test.cpu().detach().numpy(), y_test_pred.cpu().detach().numpy())
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss.item():.4f}, Test Loss: {test_loss.item():.4f}, R2 Train: {r2_train:.4f}, R2 Test: {r2_test:.4f}')

Epoch [10/50], Training Loss: 0.0001, Test Loss: 0.0002, R2 Train: 0.9900, R2 Test: 0.9836
Epoch [20/50], Training Loss: 0.0002, Test Loss: 0.0003, R2 Train: 0.9877, R2 Test: 0.9773
Epoch [30/50], Training Loss: 0.0002, Test Loss: 0.0003, R2 Train: 0.9905, R2 Test: 0.9777
Epoch [40/50], Training Loss: 0.0002, Test Loss: 0.0007, R2 Train: 0.9907, R2 Test: 0.9390
Epoch [50/50], Training Loss: 0.0001, Test Loss: 0.0003, R2 Train: 0.9910, R2 Test: 0.9786


In [9]:
model.eval()
with torch.no_grad():
    y_train_pred = model(X_train)
    y_test_pred = model(X_test)
    
y_train_pred = scaler.inverse_transform(torch.cat((y_train_pred.squeeze().unsqueeze(1), torch.zeros((y_train_pred.size(0), df_filtered.shape[1] - 1), device=device)), dim=1).cpu().numpy())[:, 0]
y_train = scaler.inverse_transform(torch.cat((y_train.unsqueeze(1), torch.zeros((y_train.size(0), df_filtered.shape[1] - 1), device=device)), dim=1).cpu().numpy())[:, 0]
y_test_pred = scaler.inverse_transform(torch.cat((y_test_pred.squeeze().unsqueeze(1), torch.zeros((y_test_pred.size(0), df_filtered.shape[1] - 1), device=device)), dim=1).cpu().numpy())[:, 0]
y_test = scaler.inverse_transform(torch.cat((y_test.unsqueeze(1), torch.zeros((y_test.size(0), df_filtered.shape[1] - 1), device=device)), dim=1).cpu().numpy())[:, 0]

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(y_train))),
    y=y_train.flatten(),
    mode='lines',
    name='Train Data',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=list(range(len(y_train))),
    y=y_train_pred.flatten(),
    mode='lines',
    name='Train Prediction',
    line=dict(color='cyan', dash='dot')
))

fig.add_trace(go.Scatter(
    x=list(range(len(y_train), len(y_train) + len(y_test))),
    y=y_test.flatten(),
    mode='lines',
    name='Test Data',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=list(range(len(y_train), len(y_train) + len(y_test))),
    y=y_test_pred.flatten(),
    mode='lines',
    name='Test Prediction',
    line=dict(color='red', dash='dot')
))

fig.update_layout(
    title="Train and Test Data vs Predictions",
    xaxis_title="Index",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()